# Reverse2 - Volatility

### 1er étape :  Nom de la machine

Nous allons dans un premier temps executer une commande pour avoir les informations générales de la machine.

Pas beaucoup d'information sur notre nom de machine avec cette commande. On sait que le nom de la machine est contenue dans le registre et plus précisemment dans \REGISTRY\MACHINE\SYSTEM\

In [ ]:
import os
def windows_info():
    os.system("vol -f dump.vmem windows.info > ./scan-trace/window-info-output.txt")
    output = os.popen("vol -f dump.vmem windows.info").read()
    print(output)

windows_info()


vol -f dump.vmem windows.registry.printkey. --offset 0xf8a000024010 --key "ControlSet001\Control\ComputerName\ComputerName"
Cette commande nous permet d'afficher le contenu de la variable **ComputerName** contenue dans le registre MACHINE\SYSTEM
Nom de la machine : WIN-LO6FAF3DTFE

In [71]:
def windows_getkey():
    os.system("vol -f dump.vmem windows.registry.printkey.PrintKey > ./scan-trace/window-getkey-output.txt")
    os.system('vol -f dump.vmem windows.registry.printkey. --offset 0xf8a000024010 --key "ControlSet001\Control\ComputerName\ComputerName" > ./scan-trace/window-getmachinename-output.txt')

windows_getkey()

### 2e étape :  Credentials

Pour ce faire on utilise hashdump qui nous récupère les hash des mots de passes de chaques sessions stocké.
Malheureusement les hash ne sont pas crackables via des outils en ligne tel que crackstation.

grâce à lsadump on peut récupérer le mot de passe de session de rick


In [ ]:
def getUserCredentials():
    os.system("vol -f dump.vmem windows.hashdump.Hashdump  > ./scan-trace/credentials/hashes.txt")
    os.system("vol -f dump.vmem windows.lsadump.Lsadump > ./scan-trace/credentials/lsadump.txt")
   
def deleteNullByte(filename):
    with open(filename, 'rb') as f:
        data = f.read()
    with open(filename, 'wb') as f:
        f.write(data.replace(b'\x00', b''))

getUserCredentials()
deleteNullByte("./scan-trace/credentials/lsadump.txt")

password : **MortyIsReallyAnOtter**

### 3e étape :  Trouver l'ip de la machine sur le réseau
On va utiliser Netscan pour trouver facilement notre IP !

On tombe en boucle sur **192.168.202.131** : L'ip de la machine

In [ ]:
def getIp():
    os.system("vol -f dump.vmem windows.netscan.NetScan > netscan.txt")

getIp()

### 4e étape : Savoir si le malware communique avec un serveur distant.
D'après le netscan, notre malware ne communique pas avec un serveur distant.

### 5e étape : The process
Pour ce faire, nous allons lancer un pstree. Cette commande nous permet d'afficher les processus en cours d'éxecution ainsi que leurs enfants.

In [ ]:
def getProcesses():
    os.system("vol -f dump.vmem windows.pstree > ./scan-trace/pstree.txt")
    os.system("vol -f dump.vmem windows.cmdline  > ./scan-trace/cmdline.txt")


getProcesses()

```
* 3820	2728	Rick And Morty	0xfa801b486b30	4	185	1	True	2018-08-04 19:32:55.000000 	N/A
** 3720	3820	vmware-tray.ex	0xfa801a4c5b30	8	147	1	True	2018-08-04 19:33:02.000000 	N/A
```
On repère vmware-tray.ex qui pourrait être notre malware et qui s'est installé lorsque notre user a lancé download.exe

In [ ]:
def processDump():
    os.system("vol -f dump.vmem -o ./ransomware-dump/process-dump windows.pslist.PsList --pid 3720 --dump")

processDump()

In [ ]:
def memoryDump():
    os.system("vol -f dump.vmem -o ./ransomware-dump/memory-dump windows.memmap --dump --pid 3720")

memoryDump()

### Etape 6 :  l'address bitcoin


On trouve facilement des strings en clair dans le dump que notre ransomware demande des bitcoins pour permettre aux utilisateurs de retrouver leur fichier. Il faut envoyer 0.16 bitcoin.

On tombe sur une string qui ressemble à une address btc : 1MmpEmebJkqXG8nQv4cjJSmxZQFVmFo63M

Le malware est donc un ransomware qui encrypt vos fichiers et demande une rançon pour les récupérer.

### Etape 7 : La cause
Pour bien comprendre ce qu'il s'est passé et ce qui a causé l'infection on peut regarder dans notre fichier cmdline.txt.

```
2748	chrome.exe	"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" --type=renderer --field-trial-handle=984,15358569600588498425,3475374789430647391,131072 --service-pipe-token=5B96B12CED256E93CD66ABC8626426FB --lang=en-US --enable-offline-auto-reload --enable-offline-auto-reload-visible-only --device-scale-factor=1 --num-raster-threads=1 

--service-request-channel-token=5B96B12CED256E93CD66ABC8626426FB --renderer-client-id=22 --mojo-platform-channel-handle=2104 /prefetch:1
3820	Rick And Morty	"C:\Torrents\Rick And Morty season 1 download.exe" 

3720	vmware-tray.ex	"C:\Users\Rick\AppData\Local\Temp\RarSFX0\vmware-tray.exe" 
```

On comprends que notre cher Rick a téléchargé le fichier depuis chrome lancer le fichier téléchargé depuis le torrent et à lancé l'executable en pensant que c'était un épisode de rick et morty..

On va donc regarder du côté des fichiers torrent si on peut trouver quelque chose.

In [ ]:
def scanAllFile():
    os.system("vol -f dump.vmem windows.filescan > filescan.txt")

scanAllFile()

On a 3 résultats de torrent dans la liste des fichiers

In [ ]:
def dumpAllTorrent():
    os.system("vol -f dump.vmem windows.dumpfiles --physaddr 0x7d8813c0")
    os.system("vol -f dump.vmem windows.dumpfiles --physaddr 0x7dae9350")
    os.system("vol -f dump.vmem windows.dumpfiles --physaddr 0x7dcbf6f0")


dumpAllTorrent()   

Dans notre deuxieme dumpfile on applique un strings et on récupère ce qui ressemble à un flag

**M3an_T0rren7_4_R!cke**

Pour le second flag, on va se tourner du côté de chrome qui a été utilisé pour télécharger nos fichiers torrent.
On récupère toutes les instances de chrome grâce à notre pstree effectué plus tôt.

On dump tous les pid

In [54]:
def memchrome():
    os.system("vol -f dump.vmem windows.memmap --dump --pid 4076")


On récupère tout nos dump de chrome et on applique un 

```
strings *.dmp | grep -A20 -B20 "\.torrent"
```

Cette commande nous permet d'afficher les 20 lignes avant et après notre match 'qui est .torrent'

Après quelques minutes de lecture de nos log on tombe sur la strings : **Hum@n_I5_Th3_Weak3s7_Link_In_Th3_Ch@inYear**

### Etape 8 : the key

On va chercher dans le code de notre ransomware et retourner sur dnSpy

Le programme génère un mot de passe aléatoire de 15 caractère au runtime.

On peut trouver ces informations facilement grâce à dnspy.

![img](images/1.png)

On se penche maintenant sur la fonction : 

```.NET
SendPassword(string password)
```

![img](images/2.png)

On remarque le mot de passe est envoyé avec le nom du pc et du user juste avant : WIN-LO6FAF3DTFE-Rick MDP

On passe un coup de strings sur le dump mémoire de notre malware

```console
do-not-try:~$ strings -e b pid.3270.dmp | grep -i 'WIN-LO6FAF3DTFE-Rick'
WIN-LO6FAF3DTFE-Rick aDOBofVYUNVnmp7
```



### 9e étape : Décrypter
Lors du passage sur dnspy. On s'est rendu compte que l'on avait à faire à HiddenTear ransomware. Après quelque recherche sur ce ransomware on trouve des programme qui permettent de decrypter des fichiers si on a le mot de passe.

On peut maintenant récupérer notre dernier flag qui se trouve dans Flag.txt

In [ ]:
def getFlag():
    os.system("vol -f dump.vmem -o ./Flag/ windows.dumpfiles --physaddr 0x7e410890 ")

getFlag()

Lorsque que l'on ouvre ce fichier sur un editeur de code on s'apercoit qu'il est rempli de NullByte. On les retire avec python.

In [63]:
def deleteNullByte(filename):
    with open(filename, 'rb') as f:
        data = f.read()
    with open(filename, 'wb') as f:
        f.write(data.replace(b'\x00', b''))

deleteNullByte('file.0x7e410890.0xfa801b0532e0.DataSectionObject.Flag.txt.dat')

On peut maintenant décrypter le contenu du fichier avec notre programme.

![img](images/4.png)

Le dernier flag : CTF{Im_Th@_B3S7_RicK_0f_Th3m_4ll}